In [187]:
import pandas as pd
import numpy as np
import nltk,spacy,re
from  nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [188]:
tweets_df = pd.read_csv('/content/Tweets-A.csv')

In [189]:
tweets_df

,tweet_id,airline_sentiment,name,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,5.703060e+17,neutral,cairdin,@VirginAmerica What @dhepburn said.,NaN,24/02/15 11:35,NaN,Eastern Time (US & Canada)
1,5.703010e+17,positive,jnardino,@VirginAmerica plus you've added commercials t...,NaN,24/02/15 11:15,NaN,Pacific Time (US & Canada)
2,5.703010e+17,neutral,yvonnalynn,@VirginAmerica I didn't today... Must mean I n...,NaN,24/02/15 11:15,Lets Play,Central Time (US & Canada)
3,5.703010e+17,negative,jnardino,@VirginAmerica it's really aggressive to blast...,NaN,24/02/15 11:15,NaN,Pacific Time (US & Canada)
4,5.703010e+17,negative,jnardino,@VirginAmerica and it's a really big bad thing...,NaN,24/02/15 11:14,NaN,Pacific Time (US & Canada)
...,...,...,...,...,...,...,...,...
14635,5.695880e+17,positive,KristenReenders,@AmericanAir thank you we got on a different f...,NaN,22/02/15 12:01,NaN,NaN
14636,5.695870e+17,negative,itsropes,@AmericanAir leaving over 20 minutes Late Flig...,NaN,22/02/15 11:59,Texas,NaN
14637,5.695870e+17,neutral,sanyabun,@AmericanAir Please bring American Airlines to...,NaN,22/02/15 11:59,"Nigeria,lagos",NaN
14638,5.695870e+17,negative,SraJackson,"@AmericanAir you have my money, you change my ...",NaN,22/02/15 11:59,New Jersey,Eastern Time (US & Canada)


In [190]:
tweets_df.columns

Index(['tweet_id', 'airline_sentiment', 'name', 'text', 'tweet_coord',
       'tweet_created', 'tweet_location', 'user_timezone'],
      dtype='object')

In [191]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tweet_id           14640 non-null  float64
 1   airline_sentiment  14640 non-null  object 
 2   name               14640 non-null  object 
 3   text               14640 non-null  object 
 4   tweet_coord        1019 non-null   object 
 5   tweet_created      14640 non-null  object 
 6   tweet_location     9907 non-null   object 
 7   user_timezone      9820 non-null   object 
dtypes: float64(1), object(7)
memory usage: 915.1+ KB


In [192]:
#Missing data percentage list
for col in tweets_df.columns:
    pct_missing = np.mean(tweets_df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

tweet_id - 0.0%
airline_sentiment - 0.0%
name - 0.0%
text - 0.0%
tweet_coord - 93.0%
tweet_created - 0.0%
tweet_location - 32.0%
user_timezone - 33.0%


In [193]:
#TweetId is primary key.......Tweet_cord has more than 80% of missing rows so dropped it
tsa_df = tweets_df.drop(columns = ['tweet_id','tweet_coord'])
tsa_df

,airline_sentiment,name,text,tweet_created,tweet_location,user_timezone
0,neutral,cairdin,@VirginAmerica What @dhepburn said.,24/02/15 11:35,NaN,Eastern Time (US & Canada)
1,positive,jnardino,@VirginAmerica plus you've added commercials t...,24/02/15 11:15,NaN,Pacific Time (US & Canada)
2,neutral,yvonnalynn,@VirginAmerica I didn't today... Must mean I n...,24/02/15 11:15,Lets Play,Central Time (US & Canada)
3,negative,jnardino,@VirginAmerica it's really aggressive to blast...,24/02/15 11:15,NaN,Pacific Time (US & Canada)
4,negative,jnardino,@VirginAmerica and it's a really big bad thing...,24/02/15 11:14,NaN,Pacific Time (US & Canada)
...,...,...,...,...,...,...
14635,positive,KristenReenders,@AmericanAir thank you we got on a different f...,22/02/15 12:01,NaN,NaN
14636,negative,itsropes,@AmericanAir leaving over 20 minutes Late Flig...,22/02/15 11:59,Texas,NaN
14637,neutral,sanyabun,@AmericanAir Please bring American Airlines to...,22/02/15 11:59,"Nigeria,lagos",NaN
14638,negative,SraJackson,"@AmericanAir you have my money, you change my ...",22/02/15 11:59,New Jersey,Eastern Time (US & Canada)


In [194]:
#Checking the occurence of each location
tsa_df["tweet_location"].value_counts()

Boston, MA                       157
New York, NY                     156
Washington, DC                   150
New York                         127
USA                              126
                                ... 
Grand Rapids,MI & Orlando, FL      1
Winter Park, FL                    1
-тнαт νσι¢є ιиѕι∂є уσυя нєα∂       1
Rijswijk ♡ The Netherlands         1
Richmond Virginia                  1
Name: tweet_location, Length: 3081, dtype: int64

In [196]:
#Boston, MA occured many times so null values can be replaced by Boston, MA in tweet_location column
tsa_df['tweet_location'].fillna('Boston, MA', inplace = True)

In [197]:
#Checking the occurence of each timezone
tsa_df["user_timezone"].value_counts()

Eastern Time (US & Canada)    3744
Central Time (US & Canada)    1931
Pacific Time (US & Canada)    1208
Quito                          738
Atlantic Time (Canada)         497
                              ... 
Prague                           1
Pretoria                         1
Saskatchewan                     1
Canberra                         1
America/Detroit                  1
Name: user_timezone, Length: 85, dtype: int64

In [198]:
#Eastern Time (US & Canada) occured many times so null values can be replaced by Eastern Time (US & Canada) in user_timezone column
tsa_df['user_timezone'].fillna('Eastern Time (US & Canada)', inplace = True)

In [200]:
#Column ['Text'] pre-processing by converting it to lower case
tsa_df.text = tsa_df.text.apply(lambda text:text.lower() if type(text) == str else text)

In [201]:
#HREF Links Removal
tsa_df.text = tsa_df.text.apply(lambda text: re.compile(r'https?://\S+|www\.\S+').sub(r'',text))

In [202]:
#HTML Text Removal
tsa_df.text = tsa_df.text.apply(lambda text: BeautifulSoup(text,'html.parser').text)

In [203]:
#HashTags Separation And Merging
tsa_df['hashtags'] = [ [ tag[1: ] for tag in i.split() if tag.startswith('#') ] for i in tsa_df.text ]

In [204]:
#@Entities Separation And Merging
tsa_df['tags'] = [ [ ent[1: ] for ent in i.split() if ent.startswith("@") ] for i in tsa_df.text ]

In [205]:
#Punctuations Removal
tsa_df.text = tsa_df.text.str.replace('[^\w\s]','')

In [206]:
#StopWords Removal
tsa_df.text = [ " ".join(([word for word in word_tokenize(i) if not word in list((stopwords.words('english')))]))  for i in tsa_df.text ]

In [207]:
#Lemmatization - Aiming to remove inflectional endings and return the base meaning or dictionary meaning of a word also known as lemma
tsa_df.text = tsa_df.text.apply(lambda text: lemmatizer.lemmatize(text))

In [208]:
tsa_df.head()

,airline_sentiment,name,text,tweet_created,tweet_location,user_timezone,hashtags,tags
0,neutral,cairdin,virginamerica dhepburn said,24/02/15 11:35,"Boston, MA",Eastern Time (US & Canada),[],"[virginamerica, dhepburn]"
1,positive,jnardino,virginamerica plus youve added commercials exp...,24/02/15 11:15,"Boston, MA",Pacific Time (US & Canada),[],[virginamerica]
2,neutral,yvonnalynn,virginamerica didnt today must mean need take ...,24/02/15 11:15,Lets Play,Central Time (US & Canada),[],[virginamerica]
3,negative,jnardino,virginamerica really aggressive blast obnoxiou...,24/02/15 11:15,"Boston, MA",Pacific Time (US & Canada),[],[virginamerica]
4,negative,jnardino,virginamerica really big bad thing,24/02/15 11:14,"Boston, MA",Pacific Time (US & Canada),[],[virginamerica]
